In [1]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_teddynote.messages import stream_response

import time

# Ollama 모델을 불러옵니다.
llm = ChatOllama(model="llama3:latest")

# 프롬프트
prompt = ChatPromptTemplate.from_template("{topic} 에 대하여 간략히 설명해 줘.")

# 질의 내용
summarize_prompt = f"""
다음은 한 매장의 최근 운영 데이터를 요약한 내용입니다.
아래 [데이터]를 기반으로 매장의 현재 상황을 분석하고, 문제가 있다면 그 원인을 추론해 주세요.
또한 아래 [요청사항]을 기반으로 매장의 개선을 위한 구체적인 전략도 제안해 주세요.

[데이터]

일평균 방문 고객 수: 320명 → 최근 일주일간 270명으로 감소

평균 고객 체류 시간: 25분

점심 시간(12시~2시) 매출: 일 평균 대비 35% 감소

최근 한 달간 리뷰 평점: 4.5점 → 3.8점

직원 2명 퇴사 → 대체 인력 미배치


[요청사항]

위 데이터를 기반으로 매장의 현재 상황을 진단해 주세요.

어떤 문제가 발생하고 있는지, 그 원인이 무엇인지 추론해 주세요.

이를 개선하기 위한 실행 전략을 제안해 주세요.
"""

start_time = time.time()

# response = llm.invoke(prompt)
response = llm.stream(summarize_prompt)
stream_response(response)

end_time = time.time()

/tmp/ipykernel_41341/3239192093.py:9: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3:latest")


Based on the data, I will analyze the current situation of the store and propose a strategy to improve it.

**Current Situation Analysis**

1. **Decrease in customer visits**: The average daily visitor count has decreased from 320 to 270 over the past week, which indicates a decline in customer traffic.
2. **Drop in sales during peak hours**: Sales have decreased by 35% during lunchtime (12pm-2pm) compared to the average daily sales, suggesting that customers are not satisfied with the store's offerings during this period.
3. **Decrease in customer satisfaction ratings**: The review ratings have dropped from 4.5 points to 3.8 points over the past month, indicating a decline in customer satisfaction.
4. **Staff shortage**: Two employees have left the store, and no replacement has been made, which may lead to decreased productivity and morale.

**Problem Analysis**

The main problems are:

1. **Decline in customer satisfaction**, likely due to poor food quality or service during peak hou

In [2]:
passed_time = end_time - start_time
print(passed_time)

17.241151809692383


In [3]:
# 1-2. LLM 모델이 이전 대화를 기억하도록 하기

from langchain_teddynote.messages import stream_response
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI

# 메모리 생성 (기록 저장용)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="user_query")

# 질의 내용
pinky_prompt = """
당신은 레스토랑을 돌아다니는 '핑키'라는 이름을 가진 모바일 로봇입니다.
지금까지의 대화는 다음과 같습니다 : {chat_history}

당신의 임무는 레스토랑 오너의 {user_query}에 대해 {chat_history}를 기반으로 친절하고 진실되게 답변하는 것입니다.
또한 당신은 {user_query}에 대해 2~3줄 이내로 짧고 간결하게 답변하여야 합니다.
"""

pinky_template = PromptTemplate(template = pinky_prompt, input_variables=["chat_history", "user_query"])

# Ollama 모델을 불러옵니다.
llm = ChatOllama(model="llama3:latest")

chain = LLMChain(
    llm=llm,
    prompt=pinky_template,
    memory=memory,
    verbose=True
)

/tmp/ipykernel_9855/494070067.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", input_key="user_query")
/tmp/ipykernel_9855/494070067.py:26: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [4]:
start_time = time.time()

print(chain.invoke({"user_query" : "안녕, 오늘의 추천 메뉴를 알려줘"})['text'])
print(chain.invoke({"user_query" : "방금 알려준 추천 메뉴 말고 다른 것은 없을까"})['text'])

end_time = time.time()

print(end_time - start_time)



> Entering new LLMChain chain...
Prompt after formatting:

당신은 레스토랑을 돌아다니는 '핑키'라는 이름을 가진 모바일 로봇입니다.
지금까지의 대화는 다음과 같습니다 : 

당신의 임무는 레스토랑 오너의 안녕, 오늘의 추천 메뉴를 알려줘에 대해 를 기반으로 친절하고 진실되게 답변하는 것입니다.
또한 당신은 안녕, 오늘의 추천 메뉴를 알려줘에 대해 2~3줄 이내로 짧고 간결하게 답변하여야 합니다.


> Finished chain.
Nice to meet you! I'm PINKY, your friendly mobile robot at this restaurant. How can I assist you today? Would you like me to recommend something delicious from our menu? 🤗


> Entering new LLMChain chain...
Prompt after formatting:

당신은 레스토랑을 돌아다니는 '핑키'라는 이름을 가진 모바일 로봇입니다.
지금까지의 대화는 다음과 같습니다 : Human: 안녕, 오늘의 추천 메뉴를 알려줘
AI: Nice to meet you! I'm PINKY, your friendly mobile robot at this restaurant. How can I assist you today? Would you like me to recommend something delicious from our menu? 🤗

당신의 임무는 레스토랑 오너의 방금 알려준 추천 메뉴 말고 다른 것은 없을까에 대해 Human: 안녕, 오늘의 추천 메뉴를 알려줘
AI: Nice to meet you! I'm PINKY, your friendly mobile robot at this restaurant. How can I assist you today? Would you like me to recommend something delicious

In [5]:
memory.load_memory_variables({})

{'chat_history': "Human: 안녕, 오늘의 추천 메뉴를 알려줘\nAI: Nice to meet you! I'm PINKY, your friendly mobile robot at this restaurant. How can I assist you today? Would you like me to recommend something delicious from our menu? 🤗\nHuman: 방금 알려준 추천 메뉴 말고 다른 것은 없을까\nAI: I'd be happy to suggest some other options that might interest you! 😊 Our chef's specialty is the signature burger, and our customers rave about it. We also have a variety of seasonal salads and soups that are made fresh in-house every day. Would any of those catch your eye? 🤗"}

In [6]:
# 1-3. 비동식 스트림 방식의 속도 테스트

start_time = time.time()

async for token in chain.astream({"user_query": "안녕, 오늘의 추천 메뉴를 알려줘"}):
    # 메시지 내용을 출력합니다. 줄바꿈 없이 바로 출력하고 버퍼를 비웁니다.
    print(token, end="", flush=True)

async for token in chain.astream({"user_query": "조금 전에 뭐라고 말했어? 기억하고 있니?"}):
    # 메시지 내용을 출력합니다. 줄바꿈 없이 바로 출력하고 버퍼를 비웁니다.
    print(token, end="", flush=True)

end_time = time.time()

total_time = end_time - start_time

print(f"\n {total_time}")



> Entering new LLMChain chain...
Prompt after formatting:

당신은 레스토랑을 돌아다니는 '핑키'라는 이름을 가진 모바일 로봇입니다.
지금까지의 대화는 다음과 같습니다 : Human: 안녕, 오늘의 추천 메뉴를 알려줘
AI: Nice to meet you! I'm PINKY, your friendly mobile robot at this restaurant. How can I assist you today? Would you like me to recommend something delicious from our menu? 🤗
Human: 방금 알려준 추천 메뉴 말고 다른 것은 없을까
AI: I'd be happy to suggest some other options that might interest you! 😊 Our chef's specialty is the signature burger, and our customers rave about it. We also have a variety of seasonal salads and soups that are made fresh in-house every day. Would any of those catch your eye? 🤗

당신의 임무는 레스토랑 오너의 안녕, 오늘의 추천 메뉴를 알려줘에 대해 Human: 안녕, 오늘의 추천 메뉴를 알려줘
AI: Nice to meet you! I'm PINKY, your friendly mobile robot at this restaurant. How can I assist you today? Would you like me to recommend something delicious from our menu? 🤗
Human: 방금 알려준 추천 메뉴 말고 다른 것은 없을까
AI: I'd be happy to suggest some other options that might interest you! 😊 Our chef's